To save on file space in the git repository, the cell outputs below have been cleared. Just click the restart/run button to populate the below cells.

In [ ]:
import hashlib
from pyHIBP import pwnedpasswords as pw

In [ ]:
"""
Query the HIBP API to see if a password has been breached.
"""
resp = pw.is_password_breached(password="secret")

if resp:
    print("Password breached! It appears {0} time(s) in the HIBP database.".format(resp))
else:
    # A non-matching password will be falsy (integer zero (0) is the return value)
    print("Nice! That password hasn't been breached.")

In [ ]:
"""
While a password is converted to a SHA-1 hash, then partially sent to the HIBP backend, 
we can also pass in a SHA-1 hash directly.
"""
hashed = hashlib.sha1("secret1".encode("utf-8")).hexdigest()
print("Using the SHA-1 hash: {0}".format(hashed))

# NB: We're actually only sending hashed[0:5] to the HIBP backend
resp = pw.is_password_breached(sha1_hash=hashed)

if resp:
    print("Password breached! It appears {0} times in the HIBP database.".format(resp))
else:
    print("Nice! That password hasn't been breached.")

In [ ]:
"""
Or, the calling application can opt to manually check the resulting partial hashes via the k-anonymity model.

If the full SHA-1 hash is provided, or the original password (which is then converted to a hash), the pwnedpasswords
module does the processing for determining if a password is breached. In fact, this is the only way we check passwords.
So, a secure password remains secure, as we only send five characters of a resulting SHA-1 hash.

See < https://www.troyhunt.com/ive-just-launched-pwned-passwords-version-2/ > for more information.
"""
hashed = hashlib.sha1("secret1".encode("utf-8")).hexdigest()
print("Using the SHA-1 partial hash: {0}".format(hashed[0:5]))

# NB: We're actually only sending hashed[0:5] to the HIBP backend
resp = pw.is_password_breached(first_5_hash_chars=hashed[0:5])

# Since we're only sending the first five characters, the calling application needs to parse through the resulting
# list, and find the remainder of the hash, if present, to determine if the password in question was breached.

print("There are {0} partial SHA-1 hashes in the response which matched the leading prefix of {1}.".format(len(resp), hashed[0:5].upper()))
print()

# Since we only passed a partial hash, we need to manually look for the corresponding hash in the response.
for partial_hash in resp:
    if hashed[5:] in partial_hash.lower():
        print("The partial hash which the HIBP Pwned Passwords API returned was: {0}".format(partial_hash.split(':')[0]))
        print("It was found used {0} time(s).".format(partial_hash.split(':')[1]))
        print("Inside the ``resp`` list, it would look like the following line...")
        print()
        print(partial_hash)